# So you Want to Calculate an Impact Risk

Here we walk you through the steps necesssary to calculate the impact risk for a specified orbit, in this case 2024 YR4.
We're going to take a few steps:

1. Fetch the state vector for 2024 YR4 from JPL Horizons
2. Run calculate impacts, it will create and propagate samples through time and check for collisions.
3. Summarize the results into an impact risk.


Before we can start, you will need to install a couple packages.

```bash
pip install adam-core[plots] adam-assist
```


In [ ]:
# Fetch the state vector from Small Bodies Database (SBDB)
from adam_core.orbits.query import query_sbdb

orbit = query_sbdb(["2024 YR4"])

orbit.to_dataframe()

In [ ]:
# We need to know how many days out to run the propagation,
# relative to when the orbit state vector is defined.
# Let's go 30 days past the possible 2032 impact of 2024 YR4
from adam_core.time import Timestamp
approx_impact_date = Timestamp.from_iso8601(["2032-12-22"], scale="tdb")
thirty_days_after_impact = approx_impact_date.add_days(30)
days_until_thirty_days_after_impact, _ = thirty_days_after_impact.difference(orbit.coordinates.time)
days_until_thirty_days_after_impact

Let's propagate the nominal orbit to about 30 days before the impact time and do a quick minimum orbit intersection distance (MOID) calculation. The propagation we do before hand is important because the MOID calculation uses 2-body dynamics (a simple Keplerian orbit) and that approximation does not hold for long periods of time.

In [ ]:
from adam_assist import ASSISTPropagator

propagator = ASSISTPropagator()
propagated_orbit = propagator.propagate_orbits(orbit, approx_impact_date.add_days(-30), covariance=True)
propagated_orbit.to_dataframe()

In [ ]:
from adam_core.dynamics.moid import calculate_perturber_moids
from adam_core.coordinates import OriginCodes

moids = calculate_perturber_moids(propagated_orbit, OriginCodes.EARTH)
moids.to_dataframe()


In [ ]:
from adam_core.constants import KM_P_AU

# Compute moid in KM
moids.select("perturber.code", "EARTH").moid[0].as_py() * KM_P_AU

In [ ]:
# Now we initialize our propagator, and pass it to our calculate_impacts function
from adam_core.dynamics.impacts import calculate_impacts, CollisionConditions
from adam_core.coordinates import Origin


# Define the collision conditions including a potential impact on the Moon
conditions = CollisionConditions.from_kwargs(
    condition_id=["Earth", "Moon"],
    collision_object=Origin.from_kwargs(code=["EARTH", "MOON"]),
    collision_distance=[6420, 1740],
    stopping_condition=[True, True],
)

variants, impacts = calculate_impacts(
    orbit,
    days_until_thirty_days_after_impact[0].as_py(),
    propagator,
    num_samples=10000,
    processes=10, # Multiprocessing speeds things up if you have the CPUs
    conditions=conditions,
)

In [ ]:
variants.to_dataframe()

In [ ]:
impacts.to_dataframe()

In [ ]:
# Alternatively, if we don't know the impact date (which is the case in reality at discovery)
from adam_core.orbits import VariantOrbits

variants_orbits = VariantOrbits.create(
    orbit,
    method="monte-carlo",
    num_samples=10000,
    seed=612,
)

variants, impacts = propagator.detect_collisions(
    variants_orbits,
    8 * 365, # Propagate for 8 years
    conditions=conditions,
    max_processes=10,
)
impacts.to_dataframe()

In [ ]:
# Now we can summarize the returns, a simple ratio in our case
# with only 1 orbit considered/
from adam_core.dynamics.impacts import calculate_impact_probabilities
ip = calculate_impact_probabilities(variants, impacts, conditions=conditions)
ip.to_dataframe()

We've recently added some visualization functionality to visualize both the risk corridor and the impact events. Let's first look at the impact corridor.

In [ ]:
from adam_core.dynamics.plots import plot_risk_corridor

fig = plot_risk_corridor(impacts, title="Risk Corridor for 2024 YR4")
fig.show()

In [12]:
from adam_core.dynamics.plots import generate_impact_visualization_data

propagation_times, propagated_best_fit_orbit, propagated_variants = generate_impact_visualization_data(
    orbit,
    variants,
    impacts,
    propagator,
    time_step=5,
    time_range=60,
    max_processes=None
)

In [ ]:
from adam_core.dynamics.plots import plot_impact_simulation

fig = plot_impact_simulation(
    propagation_times,
    propagated_best_fit_orbit, 
    propagated_variants, 
    impacts, 
    title="2024 YR4 Impact Simulation (Simulated on 2025-04-03)",
    sample_impactors=None,
    sample_non_impactors=0.1
)
fig.write_html("2024YR4_impact_simulation_20250403.html")

And there you have it. 

In [ ]:
fig